# Exploratory: Check column null percentage

Summary: <br>
This notebook is used to explore the SDG&E weather station data that was extracted via Synopic API.

Open Questions: <br>

In [1]:
import pandas as pd

pd.set_option("display.max_rows", None, "display.max_columns", None)

### Read in data

In [2]:
%%time
weather_sdge_df = pd.read_csv("../../data/raw/weather_SDGE.csv", header=[0, 1])
weather_hpwren_df = pd.read_csv("../../data/raw/weather_HPWREN.csv", header=[0, 1])
weather_scedison_df = pd.read_csv("../../data/raw/weather_SC-EDISON.csv", header=[0, 1])

CPU times: user 27.8 s, sys: 6.01 s, total: 33.8 s
Wall time: 33.8 s


### Helper functions to calculate attribute null percentage and station null percentage

In [3]:
def calc_att_null_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return dataframe with null percentage of each column given a weather data dataframe.
    """
    rows = df.shape[0]
    return pd.DataFrame(df.isna().sum() / rows * 100, columns=["null_percentage"])

In [4]:
def calc_station_null_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return dataframe with null percentage of each column for each station given a weather data dataframe.
    """
    tmp = df.set_index("Station_ID")

    # get null counts of each column for each station
    station_df = tmp.isna().groupby("Station_ID").sum()
    # update index
    # just take the station value in each list
    orig_index = station_df.index.to_list()
    new_index = [item[0] for item in orig_index]
    station_df.index = pd.Index(new_index, name="Station_ID")

    # get the total row count for each station
    counts_df = (
        tmp["Date_Time"]
        .groupby("Station_ID")
        .count()
        .rename(columns={"Unnamed: 1_level_1": "row_count"})
    )
    # update index
    # just take the station value in each list
    orig_index = counts_df.index.to_list()
    new_index = [item[0] for item in orig_index]
    counts_df.index = pd.Index(new_index, name="Station_ID")

    # calc null percentage
    station_null_per_df = station_df.copy()
    for station in counts_df.index:
        # print(station)
        # print(station_null_per_df.loc[station] / counts_df.loc[station]["row_count"])
        # print(counts_df.loc[station]["row_count"])
        station_null_per_df.loc[station] = (
            station_null_per_df.loc[station] / counts_df.loc[station]["row_count"] * 100
        )

    return station_null_per_df

### Calc null percentages for each network

#### SDGE

In [5]:
weather_sdge_df.shape

(19614476, 22)

In [6]:
%%time
calc_att_null_percentage(weather_sdge_df)

CPU times: user 6.05 s, sys: 33.8 ms, total: 6.09 s
Wall time: 6.08 s


,,null_percentage
Station_ID,Unnamed: 0_level_1,0.000000
Date_Time,Unnamed: 1_level_1,0.000000
air_temp_set_1,Celsius,0.199113
relative_humidity_set_1,%,0.156410
wind_speed_set_1,m/s,0.027709
volt_set_1,volts,63.775025
wind_gust_set_1,m/s,0.003671
wind_direction_set_1,Degrees,3.977985
dew_point_temperature_set_1d,Celsius,0.204645
wind_chill_set_1d,Celsius,97.645413


In [7]:
%%time
sdge_station_null_per_df = calc_station_null_percentage(weather_sdge_df)
sdge_station_null_per_df

CPU times: user 28.2 s, sys: 3.94 s, total: 32.1 s
Wall time: 32.1 s


Date_Time air_temp_set_1 relative_humidity_set_1  \
           Unnamed: 1_level_1        Celsius                       %   
Station_ID                                                             
ARHSD                       0       0.000000                0.001135   
AVOSD                       0       0.000000                0.000374   
BFDSD                       0       0.001140                0.001520   
BLCSD                       0       0.000000                0.000000   
BLUSD                       0       0.000000                0.001026   
BMRSD                       0       0.000000                0.000000   
BMSSD                       0       0.000376                0.001504   
BOBSD                       0       0.000000                0.000000   
BOCSD                       0       0.000000                0.000377   
BRGSD                       0       0.000000                0.001886   
BUBSD                       0       0.000000                0.000000   
CAPSD                       0       0.000000                0.000375   
CCRSD                       0       0.000000                0.000000   
CHHSD                       0       0.000000                0.001124   
CHOSD                       0       0.001194                0.001194   
CLNSD                       0       0.000000                0.000945   
COYSD                       0       0.000000                0.000752   
CRISD                       0       0.000000                0.002028   
CTMSD                       0       0.000000                0.000387   
CVXSD                       0       0.000000                0.000000   
CWDSD                       0       0.300721                0.002201   
DAMSD                       0       0.004579                0.004579   
DHVSD                       0       0.000000                0.000000   
DJZSD                       0       0.000000                0.002352   
DMHSD                       0       0.001131                0.000754   
DZRSD                       0       0.002993                0.002993   
ECJSD                       0       0.001150                0.000000   
EWNSD                       0       0.407058                0.397194   
FBKSD                       0       0.062550                0.000749   
GAVSD                       0       0.000000                0.016361   
GVYSD                       0       0.000000                0.000000   
HAUSD                       0       0.000000                0.002227   
HRPSD                       0       2.653094                0.001891   
HVLSD                       0       0.000000                0.000000   
HVYSD                       0       0.000750                0.006000   
IMTSD                       0       0.001016                0.001016   
JAMSD                       0       0.006169                0.001542   
JVRSD                       0       0.000000                0.000749   
LAGSD                       0       0.051267                0.184181   
LCKSD                       0       0.054858                0.000409   
LCMSD                       0       0.000850                0.001700   
LFRSD                       0       0.000000                0.000408   
LJHSD                       0       0.000000                0.000000   
LPTSD                       0       0.006420                0.000378   
LSVSD                       0       0.000000                0.004792   
MADSD                       0       0.000000                0.000000   
MCNSD                       0       0.000000                0.000000   
MGDSD                       0       0.000000                0.000000   
MGYSD                       0       0.000000                0.000000   
MIGSD                       0       0.001880                0.000752   
MLGSD                       0       0.000394                0.001182   
MORSD                       0       0.000000                0.108292   
MPESD                       0       0.005992                

#### HPWREN

In [8]:
weather_hpwren_df.shape

(2418771, 23)

In [9]:
%%time
calc_att_null_percentage(weather_hpwren_df)

CPU times: user 760 ms, sys: 8.37 ms, total: 769 ms
Wall time: 769 ms


,,null_percentage
Station_ID,Unnamed: 0_level_1,0.000000
Date_Time,Unnamed: 1_level_1,0.000000
pressure_set_1,Pascals,1.012126
air_temp_set_1,Celsius,1.012250
relative_humidity_set_1,%,1.012539
wind_speed_set_1,m/s,1.837007
wind_direction_set_1,Degrees,1.851353
wind_gust_set_1,m/s,1.838041
precip_accum_one_hour_set_1,Millimeters,100.000000
precip_accum_set_1,Millimeters,84.761931


In [10]:
%%time
hpwren_station_null_per_df = calc_station_null_percentage(weather_hpwren_df)
hpwren_station_null_per_df

# via trial and error there are 4 hpwren stations with > 20% null percentage for wind speed; these will be manually filtered out and reprocessed
# HP003, HP006 (100%), HP015, HP025

CPU times: user 3.49 s, sys: 322 ms, total: 3.81 s
Wall time: 3.81 s


,Date_Time,pressure_set_1,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,precip_accum_one_hour_set_1,precip_accum_set_1,volt_set_1,dew_point_temperature_set_1d,wind_chill_set_1d,wind_cardinal_direction_set_1d,sea_level_pressure_set_1d,heat_index_set_1d,wet_bulb_temperature_set_1d,altimeter_set_1d,precip_accum_five_minute_set_1,solar_radiation_set_1,precip_accum_one_minute_set_1,fuel_temp_set_1,fuel_moisture_set_1
,Unnamed: 1_level_1,Pascals,Celsius,%,m/s,Degrees,m/s,Millimeters,Millimeters,volts,Celsius,Celsius,code,Pascals,Celsius,Celsius,Pascals,Millimeters,W/m**2,Millimeters,Celsius,gm
Station_ID,,,,,,,,,,,,,,,,,,,,,,
HP001,0,1.303828,1.303828,1.303828,7.465208,7.470773,7.469978,100,85.481005,85.480608,1.303828,65.413985,7.470773,1.303828,98.103090,100,1.303828,100,100,100,100,100
HP002,0,0.639910,0.639910,0.640350,6.643691,6.645454,6.644132,100,84.769531,84.769531,0.640350,92.850816,6.645454,0.640350,93.455909,100,0.639910,100,100,100,100,100
HP004,0,1.308221,1.308623,1.308221,0.290224,0.305520,0.290224,100,87.713592,87.713592,1.309026,99.973031,0.305520,1.309428,89.520547,100,1.308221,100,100,100,100,100
HP005,0,0.857596,0.857596,0.857596,0.013425,0.020532,0.013425,100,83.935862,83.935467,0.857596,97.109363,0.020532,0.857596,92.339674,100,0.857596,100,100,100,100,100
HP007,0,1.192226,1.192226,1.192226,0.005503,0.022013,0.005896,100,85.096188,85.096188,1.192226,99.345907,0.022013,1.192226,91.625327,100,1.192226,100,100,100,100,100
HP009,0,0.912596,0.912596,0.912596,0.425801,0.513263,0.426377,100,82.673917,82.673917,0.912596,99.977559,0.513263,0.912596,94.555498,100,0.912596,100,100,100,100,100
HP010,0,0.779624,0.779624,0.779624,0.066570,0.080166,0.067039,100,83.085023,83.085491,0.779624,99.155212,0.080166,0.779624,88.803045,100,0.779624,100,100,100,100,100
HP014,0,1.047900,1.048326,1.050880,3.025376,3.025802,3.028355,100,83.605594,83.605594,1.051305,91.423561,3.025802,1.051305,97.295974,100,1.047900,100,100,100,100,100


#### SC-EDISON

In [11]:
weather_scedison_df.shape

(2995134, 13)

In [12]:
%%time
calc_att_null_percentage(weather_scedison_df)

CPU times: user 881 ms, sys: 14.6 ms, total: 896 ms
Wall time: 892 ms


,,null_percentage
Station_ID,Unnamed: 0_level_1,0.000000
Date_Time,Unnamed: 1_level_1,0.000000
air_temp_set_1,Celsius,0.112983
relative_humidity_set_1,%,0.151946
wind_speed_set_1,m/s,0.077793
wind_direction_set_1,Degrees,1.818082
wind_gust_set_1,m/s,0.081031
volt_set_1,volts,18.718194
solar_radiation_set_1,W/m**2,49.153126
dew_point_temperature_set_1d,Celsius,0.152180


In [13]:
%%time
scedison_station_null_per_df = calc_station_null_percentage(weather_scedison_df)
scedison_station_null_per_df

CPU times: user 3.85 s, sys: 174 ms, total: 4.02 s
Wall time: 4.02 s


,Date_Time,air_temp_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,volt_set_1,solar_radiation_set_1,dew_point_temperature_set_1d,wind_chill_set_1d,wind_cardinal_direction_set_1d,heat_index_set_1d
,Unnamed: 1_level_1,Celsius,%,m/s,Degrees,m/s,volts,W/m**2,Celsius,Celsius,code,Celsius
Station_ID,,,,,,,,,,,,
094SE,0,0.000000,0.000000,0.037168,2.843338,0.000000,0.000000,100.000000,0.000000,100.000000,2.843338,81.378926
152SE,0,0.000000,0.000000,0.030404,2.645181,0.000000,0.000000,100.000000,0.000000,100.000000,2.645181,99.452721
SE003,0,0.000000,0.004751,0.061763,0.043551,0.211418,28.448016,35.532505,0.004751,95.015441,0.043551,92.000950
SE007,0,0.000000,0.000000,0.087918,1.558379,0.000000,40.643536,40.660252,0.000000,100.000000,1.558379,94.178214
SE024,0,0.000000,0.000681,0.071552,0.089270,0.000000,35.295681,35.325665,0.000681,97.516798,0.089270,97.784607
SE031,0,2.124435,2.840127,0.072953,7.359424,0.001887,39.785670,66.064387,2.840755,100.000000,7.359424,93.221682
SE072,0,0.000000,0.000000,0.072141,1.497428,0.000000,35.382043,35.423844,0.000000,99.958199,1.497428,91.455694
SE073,0,0.000000,0.000000,0.086196,1.661302,0.000673,35.382295,67.109995,0.000000,99.940740,1.661302,92.947380


#### SDGE

In [14]:
tmp = weather_sdge_df.set_index("Station_ID")

In [15]:
station_df = tmp.isna().groupby("Station_ID").sum()

# update index
# just take the station value in each list
orig_index = station_df.index.to_list()
new_index = [item[0] for item in orig_index]
station_df.index = pd.Index(new_index, name="Station_ID")

station_df

Date_Time air_temp_set_1 relative_humidity_set_1  \
           Unnamed: 1_level_1        Celsius                       %   
Station_ID                                                             
ARHSD                       0              0                       3   
AVOSD                       0              0                       1   
BFDSD                       0              3                       4   
BLCSD                       0              0                       0   
BLUSD                       0              0                       1   
BMRSD                       0              0                       0   
BMSSD                       0              1                       4   
BOBSD                       0              0                       0   
BOCSD                       0              0                       1   
BRGSD                       0              0                       5   
BUBSD                       0              0                       0   
CAPSD                       0              0                       1   
CCRSD                       0              0                       0   
CHHSD                       0              0                       3   
CHOSD                       0              3                       3   
CLNSD                       0              0                       2   
COYSD                       0              0                       2   
CRISD                       0              0                       5   
CTMSD                       0              0                       1   
CVXSD                       0              0                       0   
CWDSD                       0            683                       5   
DAMSD                       0              2                       2   
DHVSD                       0              0                       0   
DJZSD                       0              0                       2   
DMHSD                       0              3                       2   
DZRSD                       0              8                       8   
ECJSD                       0              3                       0   
EWNSD                       0           1073                    1047   
FBKSD                       0            167                       2   
GAVSD                       0              0                       7   
GVYSD                       0              0                       0   
HAUSD                       0              0                       1   
HRPSD                       0           7016                       5   
HVLSD                       0              0                       0   
HVYSD                       0              2                      16   
IMTSD                       0              1                       1   
JAMSD                       0             16                       4   
JVRSD                       0              0                       2   
LAGSD                       0            135                     485   
LCKSD                       0            134                       1   
LCMSD                       0              2                       4   
LFRSD                       0              0                       1   
LJHSD                       0              0                       0   
LPTSD                       0             17                       1   
LSVSD                       0              0                      11   
MADSD                       0              0                       0   
MCNSD                       0              0                       0   
MGDSD                       0              0                       0   
MGYSD                       0              0                       0   
MIGSD                       0              5                       2   
MLGSD                       0              1                       3   
MORSD                       0              0                     289   
MPESD                       0             16                

## Check wind direction nulls

Is wind direction null only when wind speed is 0? -> Primarily yes. Wind speed=0 accounts for 99.99% of null wind directions.

In [16]:
weather_sdge_df[weather_sdge_df["wind_direction_set_1"]["Degrees"].isna()][
    "wind_speed_set_1"
]["m/s"].value_counts(normalize=True) * 100

0.000    99.987056
0.468     0.000384
1.199     0.000256
1.235     0.000256
1.980     0.000256
0.580     0.000128
0.741     0.000128
0.273     0.000128
1.451     0.000128
0.980     0.000128
1.460     0.000128
0.607     0.000128
1.394     0.000128
1.020     0.000128
1.000     0.000128
1.152     0.000128
0.319     0.000128
2.420     0.000128
0.590     0.000128
0.082     0.000128
0.210     0.000128
0.905     0.000128
1.955     0.000128
1.327     0.000128
0.700     0.000128
0.360     0.000128
2.090     0.000128
2.550     0.000128
4.970     0.000128
4.522     0.000128
0.268     0.000128
7.259     0.000128
0.823     0.000128
4.491     0.000128
2.269     0.000128
0.561     0.000128
1.970     0.000128
2.768     0.000128
1.163     0.000128
0.535     0.000128
0.653     0.000128
0.952     0.000128
2.450     0.000128
2.710     0.000128
1.550     0.000128
3.730     0.000128
2.546     0.000128
1.759     0.000128
4.337     0.000128
1.307     0.000128
2.896     0.000128
2.480     0.000128
2.380     0.

In [17]:
weather_hpwren_df[weather_hpwren_df["wind_direction_set_1"]["Degrees"].isna()][
    "wind_speed_set_1"
]["m/s"].value_counts(normalize=True) * 100

0.0    100.0
Name: m/s, dtype: float64

In [18]:
weather_scedison_df[weather_scedison_df["wind_direction_set_1"]["Degrees"].isna()][
    "wind_speed_set_1"
]["m/s"].value_counts(normalize=True) * 100

0.000    99.904507
1.821     0.003673
1.510     0.001836
2.400     0.001836
0.350     0.001836
0.442     0.001836
0.977     0.001836
1.595     0.001836
1.929     0.001836
1.873     0.001836
0.777     0.001836
1.420     0.001836
0.314     0.001836
1.183     0.001836
0.823     0.001836
0.590     0.001836
0.770     0.001836
0.833     0.001836
0.895     0.001836
0.450     0.001836
0.130     0.001836
2.819     0.001836
0.406     0.001836
0.262     0.001836
0.320     0.001836
2.900     0.001836
1.710     0.001836
0.900     0.001836
0.800     0.001836
0.864     0.001836
0.268     0.001836
0.010     0.001836
0.210     0.001836
0.310     0.001836
0.920     0.001836
0.499     0.001836
1.214     0.001836
0.746     0.001836
0.952     0.001836
2.433     0.001836
1.940     0.001836
0.797     0.001836
4.040     0.001836
1.271     0.001836
2.670     0.001836
3.611     0.001836
0.422     0.001836
1.530     0.001836
1.620     0.001836
0.440     0.001836
0.093     0.001836
2.510     0.001836
Name: m/s, d